# モジュールの準備

## 基本モジュール

In [1]:
from pprint import pprint
from datetime import date
import pandas as pd
import re

## スクレイピング関係

In [2]:
# スクレイピング関連
from time import sleep
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

---

# ポケモンの情報

In [13]:
# 名前、別名、図鑑番号、高さ、重さ
def get_pk_nomal_data(data):
    del_list = list()
    for d in data:
        k = re.search(r'(ガラル|ヨロイ島|カンムリ|アローラ)No\.\w*', d)
        if k:
            del_list.append(d)
        if d == "":
            data.remove(d)

    for d in del_list:
        data.remove(d)

    return [data[0], data[1], data[2].replace("全国", "").replace("ぜんこく", "").replace(" ", ""), data[3].replace("高さ ", ""), data[5]]

# タイプ
def get_type(browser):
    typea = browser.find_element(By.CLASS_NAME, "type")
    typeb = typea.find_elements(By.TAG_NAME, "img")
    type_list = list()
    for t in typeb:
        ty = t.get_attribute("alt")
        type_list.append(ty)
    
    if len(type_list) == 1:
        type_list.append("-")
    return type_list

def data_split(data):
    return data.split("  ")[1].split("(")[0]

def del_friend(data):
    for d in data:
        if re.match(r'仲間呼びやすさ\w*', d):
            data.remove(d)
    return data

# タマゴグループ
def get_egg(data):
    egg_data = data[32].replace("タマゴグループ ", "").split(" / ")
    if len(egg_data) == 1:
        egg_data.append("-")
    return egg_data

def char_split(data):
    return data.split(" ")[0].replace("*", "")

# 特性
def get_char(data):
    data = data[1:]
    index = data.index("◆")
    data1 = data[:index]
    data2 = data[index+1]

    if len(data1) == 1:
        data1.append("-")
    if data2 == "なし":
        data2 = "-"
    return data1 + [data2]

# 世代
def get_gene(num):
    num = int(num.split(".")[1])
    if num <= 151:
        return 1
    elif num <= 251:
        return 2
    elif num <= 386:
        return 3
    elif num <= 493:
        return 4
    elif num <= 649:
        return 5
    elif num <= 721:
        return 6
    elif num <= 809:
        return 7
    else:
        return 8

In [14]:
def get_Pokemon(browser):
    pk_data_all = list()
    df_char_list = list()
    # 名前・別名・タイプ
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.ID, "base_anchor")))
    pk_data1 = browser.find_element(By.ID, "base_anchor")
    pk_data1 = pk_data1.text.split("\n")
    nomal_data = get_pk_nomal_data(pk_data1)
    type_data = get_type(browser)

    # 種族値・タマゴグループ・特性
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.ID, "stats_anchor")))
    pk_data2 = browser.find_element(By.ID, "stats_anchor")
    pk_data2 = pk_data2.text.split("\n")
    pk_data2 = del_friend(pk_data2)

    Race_value = list(map(data_split, pk_data2[1:8]))
    Race_value[6] = Race_value[6].split(" / ")[1]
    Egg_type = get_egg(pk_data2)
    char_list = list(map(char_split, pk_data2[33:39]))
    char_list = get_char(char_list)

    df_char = list(map(lambda x: x.replace("*", "").split(" "), pk_data2[33:39]))
    df_char_list.append(df_char[1])
    df_char_list.append(df_char[3])

    sleep(1)
    pk_data_all.extend(nomal_data)
    pk_data_all.extend(type_data)
    pk_data_all.extend(Race_value)
    pk_data_all.extend(Egg_type)
    pk_data_all.extend(char_list)
    pk_data_all.append(get_gene(nomal_data[2]))

    return pk_data_all, df_char_list

---

In [15]:
options = webdriver.ChromeOptions()
# Chrome を立ち上げて確認したい場合は次の行をコメント化
options.add_argument('--headless')

columns = ["名前", "別名", "図鑑番号", "高さ", "重さ", "タイプ1", "タイプ2",  "H", "A", "B", "C", "D", "S", "合計値", "タマゴ1", "タマゴ2", "特性1", "特性2", "夢特性", "世代"]
Pokemon_Picture_book = list()
df_char_list = list()
error_list = list()

# ブラウザの立ち上げ
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)

for i in range(200, 210):
    try:
        print(i+1, "匹目")
        sleep(1)
        url = f"https://yakkun.com/sm/zukan/n{i+1}"
        browser.get(url)
        sleep(1)
        pk_data_all = get_Pokemon(browser)[0]
        df_char_list.extend(get_Pokemon(browser)[1])
        Pokemon_Picture_book.append(pk_data_all)
    except:
        error_list.append(i+1)


for i in range(850, 860):
    try:
        print(i+1, "匹目")
        sleep(1)
        url = f"https://yakkun.com/swsh/zukan/n{i+1}"
        browser.get(url)
        sleep(1)
        pk_data_all = get_Pokemon(browser)[0]
        df_char_list.extend(get_Pokemon(browser)[1])
        Pokemon_Picture_book.append(pk_data_all)
    except:
        error_list.append(i+1)

browser.quit()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\kny16\.wdm\drivers\chromedriver\win32\98.0.4758.102\chromedriver.exe] found in cache
C:\Users\kny16\AppData\Local\Temp\ipykernel_7968\4226566122.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)


201 匹目
202 匹目
203 匹目
204 匹目
205 匹目
206 匹目
207 匹目
208 匹目
209 匹目
210 匹目
851 匹目
852 匹目
853 匹目
854 匹目
855 匹目
856 匹目
857 匹目
858 匹目
859 匹目
860 匹目


In [16]:
df_pokemon = pd.DataFrame(Pokemon_Picture_book, columns=columns)
df_pokemon = df_pokemon.set_index("図鑑番号")
df_pokemon = df_pokemon[["名前", "別名", "世代", "タイプ1", "タイプ2", "特性1", "特性2", "夢特性", "H", "A", "B", "C", "D", "S", "合計値", "高さ", "重さ", "タマゴ1", "タマゴ2"]]

In [17]:
# df_pokemon.to_csv("pokemon_picturebook.csv", encoding="cp932")
display(df_pokemon)

,名前,別名,世代,タイプ1,タイプ2,特性1,特性2,夢特性,H,A,B,C,D,S,合計値,高さ,重さ,タマゴ1,タマゴ2
図鑑番号,,,,,,,,,,,,,,,,,,,
No.201,アンノーン,シンボルポケモン,2,エスパー,-,ふゆう,-,-,48,72,48,72,48,48,336,0.5m,5.0kg,タマゴ未発見,-
No.202,ソーナンス,がまんポケモン,2,エスパー,-,かげふみ,-,テレパシー,190,33,58,33,58,33,405,1.3m,28.5kg,不定形,-
No.203,キリンリキ,くびながポケモン,2,ノーマル,エスパー,せいしんりょく,はやおき,そうしょく,70,80,65,90,65,85,455,1.5m,41.5kg,陸上,-
No.204,クヌギダマ,みのむしポケモン,2,むし,-,がんじょう,-,ぼうじん,50,65,90,35,35,15,290,0.6m,7.2kg,虫,-
No.205,フォレトス,みのむしポケモン,2,むし,はがね,がんじょう,-,ぼうじん,75,90,140,60,60,40,465,1.2m,125.8kg,虫,-
No.206,ノコッチ,つちへびポケモン,2,ノーマル,-,てんのめぐみ,にげあし,びびり,100,70,70,65,65,45,415,1.5m,14.0kg,陸上,-
No.207,グライガー,とびさそりポケモン,2,じめん,ひこう,かいりきバサミ,すながくれ,めんえき,65,75,105,35,65,85,430,1.1m,64.8kg,虫,-
No.208,ハガネール,てつへびポケモン,2,はがね,じめん,いしあたま,がんじょう,ちからずく,75,85,200,55,65,30,510,9.2m,400.0kg,鉱物,-
No.209,ブルー,ようせいポケモン,2,フェアリー,-,いかく,にげあし,びびり,60,80,50,40,40,30,300,0.6m,7.8kg,陸上,妖精


---

# 特性一覧

In [21]:
df_char_list_ = pd.DataFrame(df_char_list)
df_char_list_ = df_char_list_.rename(columns={0: "特性", 1: "効果"})
df_char_list_ = df_char_list_[(df_char_list_["特性"]!="◆")&(df_char_list_["特性"]!="なし")].sort_values("特性", ascending=True)
df_char_all = df_char_list_[["特性", "効果"]][~df_char_list_[["特性", "効果"]].duplicated()].reset_index(drop=True)
# df_char_all.to_csv("pokemon_character.csv", encoding="cp932")
display(df_char_all)

,特性,効果
0,いかく,戦闘に出たときに相手の『こうげき』ランクを1段階下げる。／レベルの低い野生のポケモンと出会い...
1,いしあたま,技『アフロブレイク』『ウッドハンマー』『じごくぐるま』『すてみタックル』『とっしん』『フレア...
2,いたずらごころ,自分の使う変化技の優先度が1だけ加算される。優先度0の変化技も優先度+1として『すばやさ』に...
3,いやしのこころ,毎ターン終了時、1/3の確率で自分以外の味方の状態異常が治る。
4,かいりきバサミ,相手の技や特性で『こうげき』ランクが下がらない。
5,かげふみ,相手は逃げたり、ポケモンを交代することができなくなる。相手が『ゴースト』タイプや特性『かげふ...
6,がんじょう,HPが最大の時、『ひんし』状態になるダメージを受けても必ず1残る。また、一撃で『ひんし』にな...
7,くだけるよろい,物理技を受けると、『ぼうぎょ』ランクが1段階下がり、『すばやさ』ランクが2段階上がる。(第6...
8,じゅうなん,『まひ』状態にならない。
9,せいしんりょく,相手の技で『ひるみ』状態にならない。なお、技『きあいパンチ』には影響しない。
